Installation des requirements stockés sur mon dépot git

In [1]:
import subprocess

def install_requirements_from_github(url):
    # Télécharger le fichier requirements.txt depuis GitHub
    subprocess.run(['curl', '-o', 'requirements.txt', url])

    # Installer les bibliothèques listées dans requirements.txt
    subprocess.run(['pip', 'install', '-r', 'requirements.txt'])

# URL du fichier requirements.txt sur GitHub
requirements_url = 'https://raw.githubusercontent.com/bourydamien/projet_scoring/master/requirements.txt'

# Appeler la fonction pour installer les bibliothèques
install_requirements_from_github(requirements_url)



imports des fonctions installées

In [2]:
pip install h2O

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2O: filename=h2o-3.46.0.5-py2.py3-none-any.whl size=265646558 sha256=87bea20139a7336b1834372ae3a86841fb63c33a9a0c327aef9f38eb185a44b2
  Stored in directory: /root/.cache/pip/wheels/1a/46/4f/9b366522399306d7849672d58aefb44c9b73378d710bde2853
Successfully built h2O


In [3]:
pip install pyngrok


In [4]:
import numpy as np
import pandas as pd
import glob
import optuna
#import cuml
#from cuml.linear_model import LogisticRegression
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier, Pool, utils

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.sklearn
import gc
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import gdown
import numpy as np
import pandas as pd
from scipy.stats import boxcox
from scipy.stats import skew
from google.colab import drive
import catboost
from catboost.utils import get_gpu_device_count
import os
import pickle
import joblib
from datetime import datetime
import json
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score


import mlflow.catboost



from catboost.utils import get_gpu_device_count

from sklearn.metrics import confusion_matrix
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

Chemin ou je dois enregistrer les modeles sur le drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

from pyngrok import ngrok

# Démarrer l'UI MLflow sur le port 5000
get_ipython().system_raw("mlflow ui --port 5000 &")

# Terminer les tunnels ngrok existants
ngrok.kill()

# authentification
NGROK_AUTH_TOKEN = "2mQhc8YiUoUK6QUqgKogSMKs4rA_3Xp1NRRb6KNvokubaWec4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Créer un tunnel HTTP sur le port 5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)


MLflow Tracking UI: https://65f9-34-105-8-242.ngrok-free.app


In [7]:

scoring_folder = '/content/drive/MyDrive/Colab Notebooks/data_scoring'

# Créer un sous-dossier avec la date et l'heure
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_folder = f'{scoring_folder}/model_{timestamp}'
os.makedirs(model_folder, exist_ok=True)

# Création du deuxième dossier avec le suffixe '_weighted'
weighted_model_folder = f'{model_folder}_weighted'
os.makedirs(weighted_model_folder, exist_ok=True)

chargement des fichiers d'entrainements et de test. ( Pour plus de facilité, je les aient stockés sur le drive google)

In [8]:
def download_data():
    # URLs des fichiers sur Google Drive
    train_url = 'https://drive.google.com/uc?id=1oElrtY8GECJGLFpcB8-P5H5nwTvfhcYk'
    test_url = 'https://drive.google.com/uc?id=1OnzEHhlJqNASnP1FHiHPnb2554agP9cZ'

    # Chemins de sauvegarde locaux
    train_output = 'train_data.csv'
    test_output = 'test_data.csv'

    # Télécharger les fichiers
    gdown.download(train_url, train_output, quiet=False)
    gdown.download(test_url, test_output, quiet=False)

# Appeler la fonction pour télécharger les données
download_data()


Downloading...
From (original): https://drive.google.com/uc?id=1oElrtY8GECJGLFpcB8-P5H5nwTvfhcYk
From (redirected): https://drive.google.com/uc?id=1oElrtY8GECJGLFpcB8-P5H5nwTvfhcYk&confirm=t&uuid=34b8a0ab-dc59-401a-bf0e-31ae1cef6dbb
To: /content/train_data.csv
100%|██████████| 166M/166M [00:07<00:00, 23.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OnzEHhlJqNASnP1FHiHPnb2554agP9cZ
To: /content/test_data.csv
100%|██████████| 26.6M/26.6M [00:00<00:00, 41.1MB/s]


Chargement des données en dataframe

In [9]:
#importation des données de train
app_train = pd.read_csv('/content/train_data.csv')
print('Training data shape: ', app_train.shape)
# Testing data features
app_test = pd.read_csv('/content/test_data.csv')
print('Testing data shape: ', app_test.shape)


Training data shape:  (307511, 122)
Testing data shape:  (48744, 121)


Création d'une métrique personnalisée

In [10]:

def weighted_cost_metric(y_true, y_pred, fp_weight=10, fn_weight=1):
    """
    Fonction de coût personnalisée qui pénalise les faux positifs (FP) avec un poids plus élevé.

    :param y_true: Labels réels.
    :param y_pred: Labels prédits.
    :param fp_weight: Poids appliqué aux faux positifs.
    :param fn_weight: Poids appliqué aux faux négatifs.
    :return: Score du coût pondéré.
    """
    # Calculer la matrice de confusion
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Calcul du coût total
    weighted_cost = (fp_weight * fp) + (fn_weight * fn)

    return weighted_cost


Feature engeneering et correction des anomalies

In [11]:
def add_domain_features_with_anomaly(train_df, test_df):
    """
    Ajoute une colonne d'anomalie pour 'DAYS_EMPLOYED' et remplace les valeurs anormales,
    puis applique les transformations sur les DataFrames de train et de test.

    Args:
        train_df (pd.DataFrame): Le DataFrame d'entraînement.
        test_df (pd.DataFrame): Le DataFrame de test.

    Returns:
        train_df (pd.DataFrame): Le DataFrame d'entraînement avec les modifications.
        test_df (pd.DataFrame): Le DataFrame de test avec les modifications.
    """

    # Ajouter une colonne 'DAYS_EMPLOYED_ANOM' (anomalie) et remplacer 365243 par NaN
    for df in [train_df, test_df]:
        df['DAYS_EMPLOYED_ANOM'] = df["DAYS_EMPLOYED"] == 365243
        df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

        # Ajouter les colonnes dérivées
        df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
        df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
        df['CREDIT_TERM'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
        df['DAYS_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    print("Nouvelles colonnes ajoutées et anomalies corrigées")

    return train_df, test_df

In [12]:
app_train, app_test = add_domain_features_with_anomaly(app_train,app_test)

Nouvelles colonnes ajoutées et anomalies corrigées


In [13]:
set(app_train.columns)-set(app_test.columns)

{'TARGET'}

In [14]:
(app_train.shape , app_test.shape)

((307511, 127), (48744, 126))

Encodage des variables catégorielles

In [15]:
import os
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def encode_categorical_variables(train_data, test_data, target_column, model_folder):
    """
    Encode les variables catégorielles en deux étapes :
    1. Utilise Label Encoding pour les colonnes avec 2 catégories ou moins.
    2. Utilise One-Hot Encoding pour toutes les autres variables catégorielles.
    Puis aligne les colonnes des jeux de données et réinsère la variable cible.

    Les objets utilisés pour l'encodage (dictionnaire de LabelEncoders, colonnes One-Hot) sont enregistrés dans `model_folder`.

    :param train_data: DataFrame des données d'entraînement
    :param test_data: DataFrame des données de test
    :param target_column: Nom de la colonne cible dans les données d'entraînement
    :param model_folder: Dossier où enregistrer les objets d'encodage
    :return: DataFrames transformés (train_data, test_data)
    """

    # Extraire la variable cible
    train_labels = train_data[target_column]

    # Initialiser le dictionnaire des LabelEncoders
    label_encoders = {}
    le_count = 0

    # Appliquer le Label Encoding pour les colonnes avec 2 catégories ou moins
    for col in train_data.columns:
        if train_data[col].dtype == 'object':
            # Si 2 catégories ou moins
            if len(train_data[col].unique()) <= 2:
                le = LabelEncoder()
                # Entraîner le label encoder sur les données d'entraînement
                le.fit(train_data[col])
                # Transformer les colonnes d'entraînement et de test
                train_data[col] = le.transform(train_data[col])
                test_data[col] = le.transform(test_data[col])

                # Ajouter le LabelEncoder au dictionnaire
                label_encoders[col] = le
                le_count += 1

    print(f'{le_count} colonnes ont été label encodées.')

    # Si des colonnes ont été label encodées, enregistrer le dictionnaire de LabelEncoders
    if label_encoders:
        with open(os.path.join(model_folder, 'label_encoders.pkl'), 'wb') as f:
            pickle.dump(label_encoders, f)

    # Appliquer le One-Hot Encoding sur toutes les autres colonnes catégorielles
    train_data = pd.get_dummies(train_data)
    test_data = pd.get_dummies(test_data)

    # Enregistrer les colonnes utilisées pour le One-Hot Encoding
    with open(os.path.join(model_folder, 'one_hot_columns.pkl'), 'wb') as f:
        pickle.dump(train_data.columns.tolist(), f)

    # Aligner les colonnes des jeux de données d'entraînement et de test
    train_data, test_data = train_data.align(test_data, join='inner', axis=1)
    # Enregistrer les colonnes alignées (une seule fois)
    aligned_columns = {
    'aligned_columns': train_data.columns.tolist()  # train_data et test_data ont les mêmes colonnes ici
      }
    with open(os.path.join(model_folder, 'aligned_columns.json'), 'w') as f:
      json.dump(aligned_columns, f)
    # Réinsérer la colonne cible dans le jeu d'entraînement
    train_data[target_column] = train_labels

    print('Forme des données d\'entraînement après One-Hot Encoding et alignement: ', train_data.shape)
    print('Forme des données de test après One-Hot Encoding et alignement: ', test_data.shape)

    return train_data, test_data




In [16]:
app_train , app_test = encode_categorical_variables(app_train, app_test, "TARGET", model_folder)

3 colonnes ont été label encodées.
Forme des données d'entraînement après One-Hot Encoding et alignement:  (307511, 245)
Forme des données de test après One-Hot Encoding et alignement:  (48744, 244)


In [17]:
(app_train.shape , app_test.shape)

((307511, 245), (48744, 244))

Transformation mathématique des colonnes avec un boxcox si les données sont asymétriques

In [18]:
def apply_boxcox_transformation(train_data, test_data, skew_threshold=0.75, save_path=model_folder):
    """
    Applique la transformation Box-Cox sur les colonnes numériques du DataFrame qui sont asymétriques (skewed)
    dans les jeux d'entraînement et de test, et enregistre les informations utiles (colonnes transformées et lambdas).

    :param train_data: DataFrame d'entraînement
    :param test_data: DataFrame de test
    :param skew_threshold: Seuil de skewness pour décider de l'application de la transformation Box-Cox
    :param save_path: Chemin où enregistrer les informations du modèle (colonnes et lambdas)
    :return: DataFrames transformés (train_data, test_data) et liste des colonnes transformées avec lambda
    """

    # Sélectionner les colonnes numériques dans les deux datasets
    numeric_cols = train_data.select_dtypes(include=[np.number]).columns

    # Initialiser une liste pour garder la trace des colonnes transformées
    transformed_cols = []
    lambda_params = {}

    # Appliquer la transformation sur chaque colonne numérique
    for col in numeric_cols:
        # Vérifier que toutes les valeurs dans les deux jeux de données sont strictement positives
        if (train_data[col] > 0).all() and (test_data[col] > 0).all():
            # Calculer la skewness sur le jeu d'entraînement
            col_skew = skew(train_data[col])

            # Appliquer Box-Cox si l'asymétrie dépasse le seuil
            if abs(col_skew) > skew_threshold:
                # Appliquer la transformation Box-Cox sur le jeu d'entraînement
                train_data[col], fitted_lambda = boxcox(train_data[col])

                # Appliquer la transformation Box-Cox sur le jeu de test avec le même lambda
                test_data[col] = boxcox(test_data[col], lmbda=fitted_lambda)

                # Ajouter la colonne transformée et le lambda à la liste
                transformed_cols.append(col)
                lambda_params[col] = fitted_lambda

    # Afficher les colonnes transformées et les paramètres lambda utilisés
    print(f"Colonnes transformées avec Box-Cox : {transformed_cols}")
    print(f"Paramètres lambda utilisés : {lambda_params}")

    # Sauvegarder les colonnes transformées et les paramètres lambda dans des fichiers JSON
    with open(f'{save_path}/transformed_columns.json', 'w') as f:
        json.dump(transformed_cols, f)

    with open(f'{save_path}/lambda_params.json', 'w') as f:
        json.dump(lambda_params, f)

    return train_data, test_data, transformed_cols, lambda_params

In [19]:
train_data, test_data, transformed_cols, lambda_params = apply_boxcox_transformation(app_train,app_test, save_path=model_folder)

Colonnes transformées avec Box-Cox : ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'REGION_POPULATION_RELATIVE', 'CREDIT_INCOME_PERCENT']
Paramètres lambda utilisés : {'AMT_INCOME_TOTAL': -0.09313337858337999, 'AMT_CREDIT': 0.19208190760257246, 'REGION_POPULATION_RELATIVE': 0.26957001547446924, 'CREDIT_INCOME_PERCENT': 0.13555837759354478}


In [20]:
(app_train.shape , app_test.shape)

((307511, 245), (48744, 244))

In [21]:
correlation_matrix = train_data.corr()

# Extraire les corrélations avec la variable 'TARGET'
target_correlation = correlation_matrix['TARGET']

# Filtrer les corrélations avec un seuil
threshold = 0.9
high_corr_with_target = target_correlation[abs(target_correlation) > threshold]

In [22]:
print(len(high_corr_with_target))

1


In [23]:
print(high_corr_with_target)

TARGET    1.0
Name: TARGET, dtype: float64


Normalisation des données

In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

def normalize_train_test(train_data, test_data, save_path=model_folder):
    """
    Normalise les données d'entraînement et de test en ajustant le scaler sur les données d'entraînement
    et enregistre les paramètres de standardisation pour réutilisation dans le pipeline d'inférence.

    :param train_data: DataFrame d'entraînement
    :param test_data: DataFrame de test
    :param save_path: Chemin où enregistrer le scaler et les informations de normalisation
    :return: DataFrames normalisés (train_features_scaled, test_features_scaled)
    """

    # Séparer la colonne TARGET des données d'entraînement
    train_target = train_data['TARGET']
    train_features = train_data.drop(columns=['TARGET'])

    # Initialiser le scaler
    scaler = StandardScaler()

    # Ajuster le scaler sur les données d'entraînement et transformer les données d'entraînement
    train_features_scaled = scaler.fit_transform(train_features)
    # ajout des colonnes utilisées par le scaler
    train_columns = train_features.columns.tolist()
    # Transformer les données de test
    test_features_scaled = scaler.transform(test_data)

    # Convertir les données normalisées en DataFrame
    train_features_scaled = pd.DataFrame(train_features_scaled, columns=train_features.columns)
    test_features_scaled = pd.DataFrame(test_features_scaled, columns=test_data.columns)

    # Ajouter la colonne TARGET aux données d'entraînement normalisées
    train_features_scaled['TARGET'] = train_target.values

    # Sauvegarder le scaler pour réutilisation
    scaler_path = f'{save_path}/scaler.pkl'
    joblib.dump(scaler, scaler_path)
    print(f"Scaler sauvegardé à : {scaler_path}")
    scaler_columns_path = f'{save_path}/scaler_columns.json'
    with open(scaler_columns_path, 'w') as f:
        json.dump(train_columns, f)
    return train_features_scaled, test_features_scaled





In [25]:
app_train , app_test = normalize_train_test(train_data,test_data, save_path=model_folder)

Scaler sauvegardé à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/scaler.pkl


Entrainement du modèle avec dummy classifier

In [26]:
categorical_features = app_train.select_dtypes(include=['object', 'category']).columns.tolist()
print('List of categorical features:', categorical_features)

List of categorical features: []


In [27]:


def dummy_classifier_with_mlflow_optimise(features, cat_features=None, save_path="model_folder", exp_name="dummies_classifier_exp"):
    """
    Entraîne un DummyClassifier et enregistre les résultats dans MLflow.

    :param features: DataFrame contenant les features et la colonne cible 'TARGET'.
    :param cat_features: Ignoré pour DummyClassifier mais inclus pour la cohérence.
    :param save_path: Chemin pour sauvegarder le modèle.
    :param exp_name: Nom de l'expérience MLflow.
    """

    # Séparer les features et la cible
    X = features.drop(columns=['TARGET'])
    y = features['TARGET']

    # Créer un DummyClassifier avec une stratégie simple (most_frequent)
    dummy_clf = DummyClassifier(strategy="most_frequent", random_state=42)

    # Démarrer une expérience MLflow
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name="dummy_classifier_run"):
        # Entraîner sur l'ensemble des données d'entraînement
        dummy_clf.fit(X, y)

        # Prédictions sur les données d'entraînement
        y_pred = dummy_clf.predict(X)

        # Calculer la métrique AUC
        auc_score = roc_auc_score(y, dummy_clf.predict_proba(X)[:, 1]) if len(set(y)) == 2 else None

        # Calculer l'accuracy sur l'ensemble des données
        accuracy = accuracy_score(y, y_pred)

        # Calculer la métrique de coût pondéré
        weighted_cost = weighted_cost_metric(y, y_pred, fp_weight=10)

        # Log des paramètres et métriques dans MLflow
        mlflow.log_param("strategy", "most_frequent")
        mlflow.log_metric("auc_score", auc_score if auc_score is not None else 0)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("weighted_cost", weighted_cost)

        # Créer une DataFrame avec les métriques
        metrics_df = pd.DataFrame({
            'Metric': ['AUC', 'Accuracy', 'Weighted Cost'],
            'Score': [auc_score if auc_score is not None else 0, accuracy, weighted_cost]
        })

        print(metrics_df)  # Afficher les résultats
        metrics_df.to_csv("metrics_df.csv", index=False)  # Sauvegarder la DataFrame en CSV
        mlflow.log_artifact("metrics_df.csv")

        # Créer le répertoire de sauvegarde si nécessaire
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        # Sauvegarder le modèle localement
        model_save_path = os.path.join(save_path, "dummy_classifier_model.pkl")
        joblib.dump(dummy_clf, model_save_path)

        # Logger le modèle dans MLflow
        mlflow.sklearn.log_model(dummy_clf, "dummy_model")
        mlflow.log_artifact(model_save_path)

        print(f"Dummy classifier model logged under '{exp_name}' in MLflow and saved locally at {model_save_path}.")

    return dummy_clf, metrics_df


In [28]:
dummy_classifier_with_mlflow_optimise(app_train, cat_features=categorical_features, save_path="model_folder", exp_name="Dummie_classifier")

2024/10/07 11:12:26 INFO mlflow.tracking.fluent: Experiment with name 'Dummie_classifier' does not exist. Creating a new experiment.


          Metric         Score
0            AUC      0.500000
1       Accuracy      0.919271
2  Weighted Cost  24825.000000


2024/10/07 11:12:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Dummy classifier model logged under 'Dummie_classifier' in MLflow and saved locally at model_folder/dummy_classifier_model.pkl.


(DummyClassifier(random_state=42, strategy='most_frequent'),
           Metric         Score
 0            AUC      0.500000
 1       Accuracy      0.919271
 2  Weighted Cost  24825.000000)

entrainement avec une regrression logistique H20

In [29]:
def logistic_regression_with_mlflow_optimise(features, cat_features=None, save_path="model_folder", exp_name="logistic_regression_h2o_exp"):
    """
    Entraîne une régression logistique via H2O et enregistre les résultats dans MLflow.

    :param features: DataFrame contenant les features et la colonne cible 'TARGET'.
    :param cat_features: Liste des features catégoriques (non utilisé ici, mais ajouté pour cohérence).
    :param save_path: Chemin pour sauvegarder le modèle.
    :param exp_name: Nom de l'expérience MLflow.
    """

    # Initialiser H2O
    h2o.init()

    # Convertir le DataFrame en H2OFrame
    h2o_df = h2o.H2OFrame(features)

    # Séparer les features et la cible
    X = features.columns.difference(['TARGET']).tolist()
    y = 'TARGET'

    # Créer un modèle de régression logistique via H2O
    logistic_model = H2OGeneralizedLinearEstimator(family="binomial", lambda_=0.0)

    # Démarrer une expérience MLflow
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name="logistic_regression_run"):
        # Entraîner le modèle sur l'ensemble des données
        logistic_model.train(x=X, y=y, training_frame=h2o_df)

        # Faire des prédictions sur l'ensemble d'entraînement
        predictions = logistic_model.predict(h2o_df).as_data_frame()['p1']

        # Prédictions binaires (classes) sur l'ensemble d'entraînement
        y_pred = (predictions > 0.5).astype(int)

        # Calculer les métriques
        auc_score = roc_auc_score(features['TARGET'], predictions)
        accuracy = accuracy_score(features['TARGET'], y_pred)
        weighted_cost = weighted_cost_metric(features['TARGET'], y_pred, fp_weight=10)

        # Log des paramètres et métriques dans MLflow
        mlflow.log_param("lambda", logistic_model.lambda_)
        mlflow.log_param("family", logistic_model.family)
        mlflow.log_metric("auc_score", auc_score)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("weighted_cost", weighted_cost)  # Logging weighted cost

        # Créer une DataFrame avec les métriques
        metrics_df = pd.DataFrame({
            'Metric': ['AUC', 'Accuracy', 'Weighted Cost'],
            'Score': [auc_score, accuracy, weighted_cost]
        })

        print(metrics_df)  # Afficher les résultats
        metrics_df.to_csv("metrics_df.csv", index=False)  # Sauvegarder la DataFrame en CSV
        mlflow.log_artifact("metrics_df.csv")

        # Créer le répertoire de sauvegarde si nécessaire
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        # Sauvegarder le modèle H2O localement (il sera sauvegardé sous forme binaire)
        model_save_path = os.path.join(save_path, "logistic_regression_h2o_model")
        logistic_model.save_mojo(model_save_path)

        # Logger le modèle dans MLflow
        mlflow.log_artifact(model_save_path)

        print(f"Logistic regression model logged under '{exp_name}' in MLflow and saved locally at {model_save_path}.")

    # Arrêter H2O
    h2o.shutdown(prompt=False)

    return logistic_model, metrics_df


In [30]:
logistic_model , metrics_logistique_reg = logistic_regression_with_mlflow_optimise(app_train, cat_features=categorical_features, save_path=model_folder, exp_name="logistic_regression_h2o_exp")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmps_3mv1br
  JVM stdout: /tmp/tmps_3mv1br/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmps_3mv1br/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_unknownUser_werxji
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


2024/10/07 11:15:24 INFO mlflow.tracking.fluent: Experiment with name 'logistic_regression_h2o_exp' does not exist. Creating a new experiment.


glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


          Metric         Score
0            AUC      0.767842
1       Accuracy      0.919587
2  Weighted Cost  28967.000000
Logistic regression model logged under 'logistic_regression_h2o_exp' in MLflow and saved locally at /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/logistic_regression_h2o_model.
H2O session _sid_86aa closed.


<ipython-input-29-7ce275c8b63e>:72: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)


Entrainement avec catboost

In [31]:


def catboost_model_with_mlflow_optimise(features, test_features, cat_features, n_folds=5, save_path='model_folder', exp_name="experiment_name", run="run name"):
    """
    Entraîne et teste un modèle CatBoost en utilisant la validation croisée avec suivi MLflow et optimisation Optuna.
    Enregistre le modèle, les importances des caractéristiques et les métriques dans un dossier.

    Paramètres
    ----------
    features (pd.DataFrame): DataFrame des caractéristiques d'entraînement avec la colonne TARGET.
    test_features (pd.DataFrame): DataFrame des caractéristiques de test.
    cat_features (list): Liste des noms des caractéristiques catégorielles.
    n_folds (int, optionnel): Nombre de plis à utiliser pour la validation croisée (défaut: 5).
    save_path (str, optionnel): Dossier dans lequel enregistrer le modèle et les informations.

    Retourne
    -------
    submission (pd.DataFrame): Prédictions sur les données de test.
    feature_importances (pd.DataFrame): Importances des caractéristiques du modèle.
    metrics (pd.DataFrame): Métriques AUC de validation et d'entraînement pour chaque pli.
    """

    # Configurer le suivi MLflow, les données seront stockées dans Google Drive
    mlflow.set_experiment(exp_name)

    with mlflow.start_run(run_name=run):
        # Log des informations de base
        mlflow.log_param('n_folds', n_folds)

        # Créer le dossier de sauvegarde s'il n'existe pas
        os.makedirs(save_path, exist_ok=True)

        # Extraire les identifiants
        train_ids = features['SK_ID_CURR']
        test_ids = test_features['SK_ID_CURR']

        # Extraire les étiquettes pour l'entraînement
        labels = features['TARGET']

        # Supprimer les identifiants et la cible
        features = features.drop(columns=['SK_ID_CURR', 'TARGET'])
        test_features = test_features.drop(columns=['SK_ID_CURR'])

        # Remplacer les valeurs NaN dans les caractéristiques catégorielles par 'missing'
        for cat_feature in cat_features:
            features[cat_feature].fillna('missing', inplace=True)
            test_features[cat_feature].fillna('missing', inplace=True)

        def objective(trial):
            """Fonction objectif pour l'optimisation Optuna."""
            model = CatBoostClassifier(
                iterations=trial.suggest_int('iterations', 5000, 10000),
                learning_rate=trial.suggest_loguniform('learning_rate', 0.01, 0.1),
                depth=trial.suggest_int('depth', 4, 10),
                l2_leaf_reg=trial.suggest_loguniform('l2_leaf_reg', 1, 10),
                loss_function='Logloss',
                eval_metric='AUC',
                random_seed=50,
                cat_features=cat_features,
                logging_level='Silent',
                early_stopping_rounds=100,
                task_type='GPU' if get_gpu_device_count() > 0 else 'CPU'
            )

            out_of_fold = np.zeros(features.shape[0])
            valid_scores = []
            train_scores = []

            k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

            for fold, (train_indices, valid_indices) in enumerate(k_fold.split(features)):
                print(f"Entraînement du pli {fold+1}/{n_folds}...")

                train_features, train_labels = features.iloc[train_indices], labels.iloc[train_indices]
                valid_features, valid_labels = features.iloc[valid_indices], labels.iloc[valid_indices]

                model.fit(
                    train_features, train_labels,
                    eval_set=(valid_features, valid_labels),
                    use_best_model=True
                )

                out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

                valid_auc = roc_auc_score(valid_labels, out_of_fold[valid_indices])
                train_auc = roc_auc_score(train_labels, model.predict_proba(train_features)[:, 1])
                valid_scores.append(valid_auc)
                train_scores.append(train_auc)

            overall_valid_auc = roc_auc_score(labels, out_of_fold)
            mean_train_auc = np.mean(train_scores)

            return overall_valid_auc

        # Exécuter l'optimisation Optuna
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=5)

        # Meilleurs paramètres trouvés
        best_params = study.best_params
        print(f"Meilleurs paramètres trouvés : {best_params}")

        # Log des meilleurs paramètres
        mlflow.log_params(best_params)

        # Entraîner le modèle final avec les meilleurs paramètres
        model = CatBoostClassifier(
            **best_params,
            loss_function='Logloss',
            eval_metric='AUC',
            random_seed=50,
            cat_features=cat_features,
            logging_level='Silent',
            early_stopping_rounds=100,
            task_type='GPU' if get_gpu_device_count() > 0 else 'CPU'
        )

        out_of_fold = np.zeros(features.shape[0])
        valid_scores = []
        train_scores = []
        weighted_costs = []  # Liste pour stocker les coûts pondérés de chaque pli
        accuracies = []  # Liste pour stocker l'accuracy de chaque pli
        feature_importances = pd.DataFrame()

        k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

        for fold, (train_indices, valid_indices) in enumerate(k_fold.split(features)):
            print(f"Entraînement du pli {fold+1}/{n_folds}...")

            train_features, train_labels = features.iloc[train_indices], labels.iloc[train_indices]
            valid_features, valid_labels = features.iloc[valid_indices], labels.iloc[valid_indices]

            model.fit(
                train_features, train_labels,
                eval_set=(valid_features, valid_labels),
                use_best_model=True
            )

            out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

            valid_auc = roc_auc_score(valid_labels, out_of_fold[valid_indices])
            train_auc = roc_auc_score(train_labels, model.predict_proba(train_features)[:, 1])
            valid_scores.append(valid_auc)
            train_scores.append(train_auc)

            # Calcul du coût pondéré pour ce pli
            valid_predictions = (out_of_fold[valid_indices] > 0.5).astype(int)
            weighted_cost = weighted_cost_metric(valid_labels, valid_predictions)
            weighted_costs.append(weighted_cost)  # Ajout du coût pondéré à la liste
            mlflow.log_metric(f'weighted_cost_fold_{fold+1}', weighted_cost)

            # Calcul et log de l'accuracy pour ce pli
            accuracy = accuracy_score(valid_labels, valid_predictions)
            accuracies.append(accuracy)  # Stocke l'accuracy pour chaque pli
            mlflow.log_metric(f'accuracy_fold_{fold+1}', accuracy)  # Log de l'accuracy

            # Stocker les importances des caractéristiques
            fold_importances = pd.DataFrame()
            fold_importances['feature'] = features.columns
            fold_importances['importance'] = model.get_feature_importance()
            fold_importances['fold'] = fold + 1
            feature_importances = pd.concat([feature_importances, fold_importances], axis=0)

        overall_valid_auc = roc_auc_score(labels, out_of_fold)
        mean_train_auc = np.mean(train_scores)

        # Calcul du Weighted Cost global moyen
        mean_weighted_cost = np.mean(weighted_costs)  # Calcul du coût pondéré moyen
        mlflow.log_metric('mean_weighted_cost', mean_weighted_cost)

        # Calcul de l'accuracy moyenne
        mean_accuracy = np.mean(accuracies)
        mlflow.log_metric('mean_accuracy', mean_accuracy)  # Log de l'accuracy moyenne

        # Log des scores AUC
        mlflow.log_metric('mean_train_auc', mean_train_auc)
        mlflow.log_metric('overall_valid_auc', overall_valid_auc)

        # Prédictions sur les données de test
        test_predictions = model.predict_proba(test_features)[:, 1]
        submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})

        metrics = pd.DataFrame({
            'fold': list(range(1, n_folds + 1)),
            'train_auc': train_scores,
            'valid_auc': valid_scores,
            'weighted_cost': weighted_costs,
            'accuracy': accuracies
        })

        # Sauvegarder le modèle, les FI et les métriques
        model_path = f'{save_path}/best_model.cbm'
        model.save_model(model_path)
        mlflow.log_artifact(model_path)
        print(f"Modèle CatBoost sauvegardé à : {model_path}")

        feature_importances_path = f'{save_path}/feature_importances.csv'
        feature_importances.to_csv(feature_importances_path, index=False)
        mlflow.log_artifact(feature_importances_path)
        print(f"Importances des caractéristiques sauvegardées à : {feature_importances_path}")

        metrics_path = f'{save_path}/metrics.csv'
        metrics.to_csv(metrics_path, index=False)
        mlflow.log_artifact(metrics_path)
        print(f"Métriques sauvegardées à : {metrics_path}")

    return submission, feature_importances, metrics






In [32]:
'''
# Fermer tous les tunnels Ngrok
ngrok.kill()

# Démarrer l'UI MLflow sur le port 5000
get_ipython().system_raw("mlflow ui --port 5000 &")

# Reconnecter Ngrok pour le nouveau tunnel
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)
'''

'\n# Fermer tous les tunnels Ngrok\nngrok.kill()\n\n# Démarrer l\'UI MLflow sur le port 5000\nget_ipython().system_raw("mlflow ui --port 5000 &")\n\n# Reconnecter Ngrok pour le nouveau tunnel\nngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)\nprint("MLflow Tracking UI:", ngrok_tunnel.public_url)\n'

In [33]:
import catboost
from catboost.utils import get_gpu_device_count


In [34]:
submission, fi, metrics = catboost_model_with_mlflow_optimise(app_train, app_test, categorical_features, n_folds=4, save_path=model_folder,  exp_name="scoring_catboost", run="sans le param weighted")
print('Baseline metrics')
print(metrics)

2024/10/07 11:17:42 INFO mlflow.tracking.fluent: Experiment with name 'scoring_catboost' does not exist. Creating a new experiment.
[I 2024-10-07 11:17:43,258] A new study created in memory with name: no-name-902143fc-bfb4-4ac5-9125-a51790a65592


Entraînement du pli 1/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 2/4...


Default metric period is 5 because AUC is/are not implemented for GPU


...

Entraînement du pli 3/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 4/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Modèle CatBoost sauvegardé à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/best_model.cbm
Importances des caractéristiques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/feature_importances.csv
Métriques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/metrics.csv
Baseline metrics
   fold  train_auc  valid_auc  weighted_cost  accuracy
0     1   0.825463   0.763029           6869  0.920836
1     2   0.842178   0.768767           7548  0.918676
2     3   0.822499   0.768699           7099  0.921356
3     4   0.822724   0.766161           7339  0.917765


In [35]:
fi

,feature,importance,fold
0,NAME_CONTRACT_TYPE,0.749925,1
1,FLAG_OWN_CAR,0.690380,1
2,FLAG_OWN_REALTY,0.274565,1
3,CNT_CHILDREN,0.307429,1
4,AMT_INCOME_TOTAL,1.321716,1
...,...,...,...
238,WALLSMATERIAL_MODE_Panel,0.062414,4
239,"WALLSMATERIAL_MODE_Stone, brick",0.219386,4
240,WALLSMATERIAL_MODE_Wooden,0.029181,4
241,EMERGENCYSTATE_MODE_No,0.054004,4


Test avec les class weight de catboost

In [36]:
def catboost_model_with_weights_and_mlflow_optimise(features, test_features, cat_features, n_folds=5, save_path='model_folder', exp_name="experiment_name", run="run name"):
    """
    Entraîne et teste un modèle CatBoost en utilisant la validation croisée avec suivi MLflow et optimisation Optuna.
    Enregistre le modèle, les importances des caractéristiques et les métriques dans un dossier.

    Paramètres
    ----------
    features (pd.DataFrame): DataFrame des caractéristiques d'entraînement avec la colonne TARGET.
    test_features (pd.DataFrame): DataFrame des caractéristiques de test.
    cat_features (list): Liste des noms des caractéristiques catégorielles.
    n_folds (int, optionnel): Nombre de plis à utiliser pour la validation croisée (défaut: 5).
    save_path (str, optionnel): Dossier dans lequel enregistrer le modèle et les informations.

    Retourne
    -------
    submission (pd.DataFrame): Prédictions sur les données de test.
    feature_importances (pd.DataFrame): Importances des caractéristiques du modèle.
    metrics (pd.DataFrame): Métriques AUC de validation et d'entraînement pour chaque pli.
    """

    # Configuration du nom de l'experiment
    mlflow.set_experiment(exp_name)

    with mlflow.start_run(run_name=run):
        # Log des informations de base
        mlflow.log_param('n_folds', n_folds)

        # Créer le dossier de sauvegarde s'il n'existe pas
        os.makedirs(save_path, exist_ok=True)

        # Extraire les identifiants
        train_ids = features['SK_ID_CURR']
        test_ids = test_features['SK_ID_CURR']

        # Extraire les étiquettes pour l'entraînement
        labels = features['TARGET']

        # Supprimer les identifiants et la cible
        features = features.drop(columns=['SK_ID_CURR', 'TARGET'])
        test_features = test_features.drop(columns=['SK_ID_CURR'])

        # Remplacer les valeurs NaN dans les caractéristiques catégorielles par 'missing'
        for cat_feature in cat_features:
            features[cat_feature].fillna('missing', inplace=True)
            test_features[cat_feature].fillna('missing', inplace=True)

        # Calculer les poids des classes pour gérer le déséquilibre
        class_weights = {0: len(labels) / (2 * (len(labels[labels == 0]))),
                     1: len(labels) / (2 * (len(labels[labels == 1])))}

        def objective(trial):
            """Fonction objectif pour l'optimisation Optuna."""
            model = CatBoostClassifier(
            iterations=trial.suggest_int('iterations', 5000, 10000),
            learning_rate=trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            depth=trial.suggest_int('depth', 4, 10),
            l2_leaf_reg=trial.suggest_loguniform('l2_leaf_reg', 1, 10),
            loss_function='Logloss',
            eval_metric='AUC',
            random_seed=50,
            cat_features=cat_features,
            class_weights=class_weights,  # Ajout des poids de classes
            logging_level='Silent',
            early_stopping_rounds=100,
            task_type='GPU' if get_gpu_device_count() > 0 else 'CPU'
        )

            out_of_fold = np.zeros(features.shape[0])
            valid_scores = []
            train_scores = []

            k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

            for fold, (train_indices, valid_indices) in enumerate(k_fold.split(features)):
                print(f"Entraînement du pli {fold+1}/{n_folds}...")

                train_features, train_labels = features.iloc[train_indices], labels.iloc[train_indices]
                valid_features, valid_labels = features.iloc[valid_indices], labels.iloc[valid_indices]

                model.fit(
                    train_features, train_labels,
                    eval_set=(valid_features, valid_labels),
                    use_best_model=True
                )

                out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

                valid_auc = roc_auc_score(valid_labels, out_of_fold[valid_indices])
                train_auc = roc_auc_score(train_labels, model.predict_proba(train_features)[:, 1])
                valid_scores.append(valid_auc)
                train_scores.append(train_auc)

            overall_valid_auc = roc_auc_score(labels, out_of_fold)
            mean_train_auc = np.mean(train_scores)

            return overall_valid_auc

        # Exécuter l'optimisation Optuna
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=5)

        # Meilleurs paramètres trouvés
        best_params = study.best_params
        print(f"Meilleurs paramètres trouvés : {best_params}")

        # Log des meilleurs paramètres
        mlflow.log_params(best_params)

        # Entraîner le modèle final avec les meilleurs paramètres
        model = CatBoostClassifier(
            **best_params,
            loss_function='Logloss',
            eval_metric='AUC',
            random_seed=50,
            cat_features=cat_features,
            logging_level='Silent',
            early_stopping_rounds=100,
            task_type='GPU' if get_gpu_device_count() > 0 else 'CPU'
        )

        out_of_fold = np.zeros(features.shape[0])
        valid_scores = []
        train_scores = []
        weighted_costs = []  # Liste pour stocker les coûts pondérés de chaque pli
        accuracies = []  # Liste pour stocker l'accuracy de chaque pli
        feature_importances = pd.DataFrame()

        k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=50)

        for fold, (train_indices, valid_indices) in enumerate(k_fold.split(features)):
            print(f"Entraînement du pli {fold+1}/{n_folds}...")

            train_features, train_labels = features.iloc[train_indices], labels.iloc[train_indices]
            valid_features, valid_labels = features.iloc[valid_indices], labels.iloc[valid_indices]

            model.fit(
                train_features, train_labels,
                eval_set=(valid_features, valid_labels),
                use_best_model=True
            )

            out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]

            valid_auc = roc_auc_score(valid_labels, out_of_fold[valid_indices])
            train_auc = roc_auc_score(train_labels, model.predict_proba(train_features)[:, 1])
            valid_scores.append(valid_auc)
            train_scores.append(train_auc)

            # Calcul du coût pondéré pour ce pli
            valid_predictions = (out_of_fold[valid_indices] > 0.5).astype(int)
            weighted_cost = weighted_cost_metric(valid_labels, valid_predictions)
            weighted_costs.append(weighted_cost)  # Ajout du coût pondéré à la liste
            mlflow.log_metric(f'weighted_cost_fold_{fold+1}', weighted_cost)

            # Calcul et log de l'accuracy pour ce pli
            accuracy = accuracy_score(valid_labels, valid_predictions)
            accuracies.append(accuracy)  # Stocke l'accuracy pour chaque pli
            mlflow.log_metric(f'accuracy_fold_{fold+1}', accuracy)  # Log de l'accuracy

            # Stocker les importances des caractéristiques
            fold_importances = pd.DataFrame()
            fold_importances['feature'] = features.columns
            fold_importances['importance'] = model.get_feature_importance()
            fold_importances['fold'] = fold + 1
            feature_importances = pd.concat([feature_importances, fold_importances], axis=0)

        overall_valid_auc = roc_auc_score(labels, out_of_fold)
        mean_train_auc = np.mean(train_scores)

        # Calcul du Weighted Cost global moyen
        mean_weighted_cost = np.mean(weighted_costs)  # Calcul du coût pondéré moyen
        mlflow.log_metric('mean_weighted_cost', mean_weighted_cost)

        # Calcul de l'accuracy moyenne
        mean_accuracy = np.mean(accuracies)
        mlflow.log_metric('mean_accuracy', mean_accuracy)  # Log de l'accuracy moyenne

        # Log des scores AUC
        mlflow.log_metric('mean_train_auc', mean_train_auc)
        mlflow.log_metric('overall_valid_auc', overall_valid_auc)

        # Prédictions sur les données de test
        test_predictions = model.predict_proba(test_features)[:, 1]
        submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})

        metrics = pd.DataFrame({
            'fold': list(range(1, n_folds + 1)),
            'train_auc': train_scores,
            'valid_auc': valid_scores,
            'weighted_cost': weighted_costs,
            'accuracy': accuracies
        })

        # Sauvegarder le modèle, les FI et les métriques
        model_path = f'{save_path}/best_model_weighted_class.cbm'
        model.save_model(model_path)
        mlflow.log_artifact(model_path)
        print(f"Modèle CatBoost sauvegardé à : {model_path}")

        feature_importances_path = f'{save_path}/feature_importances_weighted_class.csv'
        feature_importances.to_csv(feature_importances_path, index=False)
        mlflow.log_artifact(feature_importances_path)
        print(f"Importances des caractéristiques sauvegardées à : {feature_importances_path}")

        metrics_path = f'{save_path}/metrics_weighted_class.csv'
        metrics.to_csv(metrics_path, index=False)
        mlflow.log_artifact(metrics_path)
        print(f"Métriques sauvegardées à : {metrics_path}")

    return submission, feature_importances, metrics

In [37]:
submission_weight, fi_weight, metrics_weight = catboost_model_with_weights_and_mlflow_optimise(app_train, app_test, categorical_features, n_folds=4, save_path=model_folder, exp_name="catboost_weighted_class", run="catboost_weighted")
print('Baseline metrics with weight')
print(metrics)

2024/10/07 11:33:16 INFO mlflow.tracking.fluent: Experiment with name 'catboost_weighted_class' does not exist. Creating a new experiment.
[I 2024-10-07 11:33:16,912] A new study created in memory with name: no-name-b6194008-b50e-4f79-874b-5852f5457b7f


Entraînement du pli 1/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 2/4...


Default metric period is 5 because AUC is/are not implemented for GPU


...

Entraînement du pli 3/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 4/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Modèle CatBoost sauvegardé à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/best_model_weighted_class.cbm
Importances des caractéristiques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/feature_importances_weighted_class.csv
Métriques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/metrics_weighted_class.csv
Baseline metrics with weight
   fold  train_auc  valid_auc  weighted_cost  accuracy
0     1   0.825463   0.763029           6869  0.920836
1     2   0.842178   0.768767           7548  0.918676
2     3   0.822499   0.768699           7099  0.921356
3     4   0.822724   0.766161           7339  0.917765


test avec le dataset balanced

In [38]:
import pandas as pd

def balance_classes_subsample(df, target_col):
    """
    Effectue un sous-échantillonnage pour équilibrer les classes dans un DataFrame.

    Paramètres
    ----------
    df (pd.DataFrame): DataFrame contenant les données avec une colonne cible déséquilibrée.
    target_col (str): Nom de la colonne cible indiquant les classes.

    Retourne
    -------
    df_balanced (pd.DataFrame): DataFrame avec des classes équilibrées par sous-échantillonnage.
    """
    # Séparer les classes
    class_counts = df[target_col].value_counts()
    class_minority = class_counts.idxmin()  # La classe minoritaire
    class_majority = class_counts.idxmax()  # La classe majoritaire

    print(f"Classe minoritaire: {class_minority} ({class_counts[class_minority]} exemples)")
    print(f"Classe majoritaire: {class_majority} ({class_counts[class_majority]} exemples)")

    # Obtenir les échantillons de chaque classe
    df_minority = df[df[target_col] == class_minority]
    df_majority = df[df[target_col] == class_majority]

    # Sous-échantillonnage de la classe majoritaire pour qu'elle ait le même nombre que la classe minoritaire
    df_majority_subsampled = df_majority.sample(n=len(df_minority), random_state=50)

    # Combiner les échantillons sous-échantillonnés avec la classe minoritaire
    df_balanced = pd.concat([df_minority, df_majority_subsampled], axis=0)

    # Mélanger les données pour éviter l'ordre
    df_balanced = df_balanced.sample(frac=1, random_state=50).reset_index(drop=True)

    print(f"Nouvelle distribution des classes : \n{df_balanced[target_col].value_counts()}")

    return df_balanced


In [39]:
app_train = balance_classes_subsample(app_train, "TARGET")

Classe minoritaire: 1 (24825 exemples)
Classe majoritaire: 0 (282686 exemples)
Nouvelle distribution des classes : 
TARGET
0    24825
1    24825
Name: count, dtype: int64


In [40]:
submission_balanced, fi_balanced, metrics_balanced = catboost_model_with_mlflow_optimise(app_train, app_test, categorical_features, n_folds=4, save_path=model_folder,  exp_name="catboost with balanced dataset", run="catboost_balanced_dataset")
print('Baseline metrics')
print(metrics)

2024/10/07 11:54:08 INFO mlflow.tracking.fluent: Experiment with name 'catboost with balanced dataset' does not exist. Creating a new experiment.
[I 2024-10-07 11:54:08,700] A new study created in memory with name: no-name-64d91843-6d33-4dbe-a26f-7f7f31449c56


Entraînement du pli 1/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 2/4...


Default metric period is 5 because AUC is/are not implemented for GPU


...

Entraînement du pli 3/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Entraînement du pli 4/4...


Default metric period is 5 because AUC is/are not implemented for GPU


Modèle CatBoost sauvegardé à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/best_model.cbm
Importances des caractéristiques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/feature_importances.csv
Métriques sauvegardées à : /content/drive/MyDrive/Colab Notebooks/data_scoring/model_20241007_111048/metrics.csv
Baseline metrics
   fold  train_auc  valid_auc  weighted_cost  accuracy
0     1   0.825463   0.763029           6869  0.920836
1     2   0.842178   0.768767           7548  0.918676
2     3   0.822499   0.768699           7099  0.921356
3     4   0.822724   0.766161           7339  0.917765


In [41]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct  7 12:42:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              44W /  70W |    107MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [42]:
token = os.getenv('GITHUB_TOKEN')
print(f"Token utilisé : {token}")


Token utilisé : None
